<a href="https://colab.research.google.com/github/Ismail-Ryad/image-colourization/blob/master/Densenet121_Image_Classification_Deployed_using_Flask_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

drive  sample_data


In [8]:
from flask import Flask, render_template, request, redirect, url_for
#from get_images import get_images, get_path, get_directory
#from get_prediction import get_prediction
#from generate_html import generate_html
from torchvision import models
import json

app = Flask(__name__)

# mapping
imagenet_class_mapping = json.load(open('/content/drive/My Drive/test/Densenet121-Image-Classification-Deployed-using-Flask-master/imagenet_class_index.json'))

# Make sure to pass `pretrained` as `True` to use the pretrained weights:
model = models.densenet121(pretrained=True)
# Since we are using our model only for inference, switch to `eval` mode:
model.eval()


def get_image_class(path):
    get_images(path)
    path = get_path(path)
    images_with_tags = get_prediction(model, imagenet_class_mapping, path)
    print(images_with_tags)
    generate_html(images_with_tags)

@app.route('/')
def home():
    return render_template('/content/drive/My Drive/test/Densenet121-Image-Classification-Deployed-using-Flask-master/templates/home.html')


@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        user = request.form['search']
        get_image_class(user)
        return redirect(url_for('success', name=get_directory(user)))


@app.route('/success/<name>')
def success(name):
    return render_template('/content/drive/My Drive/test/Densenet121-Image-Classification-Deployed-using-Flask-master/templates/image_class.html')




In [9]:
# importing required libraries
import requests
from bs4 import BeautifulSoup
import os
import time

def get_directory(url):
    return "URL_" + str(url.replace("/","_"))

def get_path(url):
    return "static/URL_" + str(url.replace("/","_"))

headers = {
    'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
    }

def get_images(url):
    path = get_path(url)
    try:
        os.mkdir(path)
    except:
        pass
    response = requests.request("GET", url, headers=headers)
    data = BeautifulSoup(response.text, 'html.parser')
    images = data.find_all('img', src=True)
    print('Number of Images: ', len(images))
    # select src tag
    image_src = [x['src'] for x in images]
    # select only jp format images
    image_src = [x for x in image_src if x.endswith('.jpeg') ]
    image_count = 1
    for image in image_src:
        print(image)
        image_file_name = path+'/'+str(image_count)+'.jpeg' 
        print(image_file_name)
        with open(image_file_name, 'wb') as f:
            res = requests.get(image)
            f.write(res.content)
        image_count = image_count+1



In [10]:
import json
import pandas as pd
import io
import glob
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

def get_category(model, imagenet_class_mapping, image_path):
    with open(image_path, 'rb') as file:
        image_bytes = file.read()
    transformed_image = transform_image(image_bytes=image_bytes)
    outputs = model.forward(transformed_image)
    
    _, category = outputs.max(1)
    
    predicted_idx = str(category.item())
    return imagenet_class_mapping[predicted_idx]

def get_prediction(model, imagenet_class_mapping, path_to_directory):
    files = glob.glob(path_to_directory+'/*')
    image_with_tags = {}
    for image_file in files:
        image_with_tags[image_file] = get_category(model, imagenet_class_mapping, image_path=image_file)[1]
    return image_with_tags



In [11]:
import glob

html_string_start = """
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>

<style>
#left-bar {
  position: fixed;
  display: table-cell;
  top: 100;
  bottom: 10;
  left: 10;
  width: 35%;
  overflow-y: auto;
}

#right-bar {
  position: fixed;
  display: table-cell;
  top: 100;
  bottom: 10;
  right: 10;
  width: 45%;
  overflow-y: auto;
}

</style>
<body>

<center><h1> Image Classification </h1></center>

<div id= "left-bar" >

"""


html_string_end = """

</body>
</html>

"""



In [12]:

def get_picture_html(path, tag):
    image_html = """<p> {tag_name} </p> <picture> <img src= "../{path_name}"  height="300" width="400"> </picture>"""
    return image_html.format(tag_name=tag, path_name=path)

def get_count_html(category, count):
    count_html = """<li> {category_name} : {count_} </li>"""
    return count_html.format(category_name = category, count_ = count)


def get_value_count(image_class_dict):
    count_dic = {}
    for category in image_class_dict.values():
        if category in count_dic.keys():
            count_dic[category] = count_dic[category]+1
        else:
            count_dic[category] = 1
    return count_dic


def generate_html(image_class_dict):
    picture_html = ""
    count_html = ""
    
    for image in image_class_dict.keys():
        picture_html += get_picture_html(path=image, tag= image_class_dict[image])
        
    value_counts = get_value_count(image_class_dict)
    
    for value in value_counts.keys():
        count_html += get_count_html(value, value_counts[value])
        
    file_content = html_string_start + picture_html + """</div> <div id= "right-bar" >""" + count_html+ "</div>" +html_string_end
    with open('templates/image_class.html', 'w') as f:
        f.write(file_content)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
